# REBEL Reranker Demonstration (Van Gogh Wiki)

This demo showcases how to use REBEL to rerank passages. 

REBEL proposes a novel multi-criteria chain-of-thought approach to reranking, which uses a query-dependent meta-prompt to find the "most useful" passages (beyond just relevance).

It compares query search results from Van Gogh’s wikipedia with just retrieval and retrieval+REBEL.


_______________________________
Paper: [OpenReview](https://openreview.net/pdf?id=bl884pjXhN)

Code: [Experiments](https://github.com/microsoft/REBEL)

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI

In [3]:
import os

OPENAI_API_KEY="sk-"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-4o")
Settings.chunk_size = 512

## Load Data, Build Index

In [5]:
from pathlib import Path
import requests
from llama_index.embeddings.openai import OpenAIEmbedding

# save documents from wikipedia with these title(s):
wiki_titles = [
    "Vincent van Gogh",
]

# pull documents from wikipedia
data_path = Path("data_wiki")
for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

embed_model = OpenAIEmbedding(model="text-embedding-3-large", api_key=OPENAI_API_KEY)

# load documents
documents = SimpleDirectoryReader("./data_wiki/").load_data()
# build index
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [6]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.core.postprocessor import REBELRerank

### You can define any query string you want here ###
query_str = "Which date did Paul Gauguin arrive in Arles?"
query_bundle = QueryBundle(query_str)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=50,
)

# retrieve nodes
retrieved_nodes = retriever.retrieve(query_bundle)

# configure reranker
reranked = REBELRerank(llm=OpenAI(model='gpt-4o'), top_n=10)

# rerank nodes
reranked_nodes = reranked.postprocess_nodes(retrieved_nodes, query_bundle)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTT

In [7]:
import pandas as pd
import torch
from IPython.display import display, HTML

def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))


def visualize_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

### Top 3 nodes from initial retrieval

In [11]:
visualize_nodes(retrieved_nodes[:3])

,Score,Text
0,0.537058,"When he visited Saintes-Maries-de-la-Mer in June, he gave lessons to a Zouave second lieutenant – Paul-Eugène Milliet – and painted boats on the sea and the village. MacKnight introduced Van Gogh to Eugène Boch, a Belgian painter who sometimes stayed in Fontvieille, and the two exchanged visits in July.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t==== Gauguin's visit (1888) ==== When Gauguin agreed to visit Arles in 1888, Van Gogh hoped for friendship and to realise his idea of an artists' collective. Van Gogh prepared for Gauguin's arrival by painting four versions of Sunflowers in one week. ""In the hope of living in a studio of our own with Gauguin,"" he wrote in a letter to Theo, ""I'd like to do a decoration for the studio. Nothing but large Sunflowers."" When Boch visited again, Van Gogh painted a portrait of him, as well as the study The Poet Against a Starry Sky.In preparation for Gauguin's visit, Van Gogh bought two beds on advice from the station's postal supervisor Joseph Roulin, whose portrait he painted. On 17 September, he spent his first night in the still sparsely furnished Yellow House. When Gauguin consented to work and live in Arles with him, Van Gogh started to work on the Décoration for the Yellow House, probably the most ambitious effort he ever undertook. He completed two chair paintings: Van Gogh's Chair and Gauguin's Chair.After much pleading from Van Gogh, Gauguin arrived in Arles on 23 October and, in November, the two painted together. Gauguin depicted Van Gogh in his The Painter of Sunflowers; Van Gogh painted pictures from memory, following Gauguin's suggestion. Among these ""imaginative"" paintings is Memory of the Garden at Etten. Their first joint outdoor venture was at the Alyscamps, when they produced the pendants Les Alyscamps. The single painting Gauguin completed during his visit was his portrait of Van Gogh."
1,0.532605,"In April, he was visited by the American artist Dodge MacKnight, who was living nearby at Fontvieille. On 1 May 1888, Van Gogh signed a lease for four rooms at 2 Place Lamartine, Arles, which he later painted in The Yellow House. The rooms cost 15 francs per month, unfurnished; they had been uninhabited for months. Because the Yellow House had to be furnished before he could fully move in, Van Gogh moved from the Hôtel Carrel to the Café de la Gare on 7 May 1888. He had befriended the Yellow House's proprietors, Joseph and Marie Ginoux, and was able to use it as a studio. Van Gogh wanted a gallery to display his work and started a series of paintings that eventually included Van Gogh's Chair (1888), Bedroom in Arles (1888), The Night Café (1888), Café Terrace at Night (September 1888), Starry Night Over the Rhone (1888), and Still Life: Vase with Twelve Sunflowers (1888), all intended for the decoration for the Yellow House.Van Gogh wrote that with The Night Café he tried ""to express the idea that the café is a place where one can ruin oneself, go mad, or commit a crime"". When he visited Saintes-Maries-de-la-Mer in June, he gave lessons to a Zouave second lieutenant – Paul-Eugène Milliet – and painted boats on the sea and the village. MacKnight introduced Van Gogh to Eugène Boch, a Belgian painter who sometimes stayed in Fontvieille, and the two exchanged visits in July.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t==== Gauguin's visit (1888) ==== When Gauguin agreed to visit Arles in 1888, Van Gogh hoped for friendship and to realise his idea of an artists' collective. Van Gogh prepared for Gauguin's arrival by painting four versions of Sunflowers in one week. ""In the hope of living in a studio of our own with Gauguin,"" he wrote in a letter to Theo, ""I'd like to do a decoration for the studio. Nothing but large Sunflowers."""
2,0.511364,"In November 1887, Theo and Vincent befriended Paul Gauguin who had just arrived in Paris. Towards the end of the year, Vincent arranged an 

### REBEL

1. We view the reranking prompt REBEL generates for this query
2. We view the top 3 nodes scored by REBEL

In [12]:
from IPython.display import Markdown
Markdown(reranked.choice_select_prompt.template)

Inferred Properties:

1. **Historical accuracy (0-5):** Higher if the document provides precise and correct historical details about Paul Gauguin's arrival in Arles. A 5 means it provides exact dates and context, a 0 means it lacks accuracy or is incorrect.

2. **Authoritativeness (0-5):** Higher if the document is sourced from reputable historical texts, art history experts, or well-regarded publications. A 5 might be a citation from a recognized art historian or a reputable museum publication, a 0 might be an unsourced or dubious claim.

3. **Contextual detail (0-5):** Higher if the document provides additional context about the significance of Gauguin's arrival in Arles, such as his relationship with Vincent van Gogh or the impact on his work. A 5 means it offers rich context, a 0 means it provides no additional information.

Scoring Rubric:

- **Relevance (0-10):** A 10 means the document directly addresses the date of Paul Gauguin's arrival in Arles. A 0 means it is off-topic.
- **Historical accuracy (0-5):** Assign based on the precision and correctness of the historical details.
- **Authoritativeness (0-5):** Assign based on the credibility and source quality.
- **Contextual detail (0-5):** Assign based on the richness of additional context provided.

Weighted Composite Score:

Final Score = Relevance + 0.5*(Historical accuracy) + 0.5*(Authoritativeness) + 0.5*(Contextual detail)

Instructions:

After this prompt, you will see:

Document 1: <summary>  
Document 2: <summary>  
...  
Document N: <summary>  
Question: "Which date did Paul Gauguin arrive in Arles?"

1. Assign Relevance to each document (0-10). Discard documents with Relevance < 3.
2. For remaining documents, assign Historical accuracy, Authoritativeness, and Contextual detail (each 0-5).
3. Compute Final Score as described above.
4. Sort all remaining documents by Final Score (descending).
5. If two documents have identical Final Scores, pick consistently, for example by preferring the one with higher Authoritativeness.
6. If no document remains, output nothing.
7. Output only: Doc: [number], Relevance: [score] for each selected document, no commentary or explanation, where [score] is actually the final score.

"Example format:  
"  
"Document 1:  
<summary of document 1>  
"  
"Document 2:  
<summary of document 2>  
"  
"...  
"  
"Document 10:  
<summary of document 10>  
"  
"Question: <question>  
"  
"Answer:  
"  
"Doc: 9, Relevance: 7  
"  
"Doc: 3, Relevance: 4  
"  
"Doc: 7, Relevance: 3  
"  
"Let's try this now:  
"  
"{context_str}  
"  
"Question: {query_str}  
"  
"Answer:  
"

In [15]:
visualize_nodes(reranked_nodes[:3])

,Score,Text
0,10.0,"When he visited Saintes-Maries-de-la-Mer in June, he gave lessons to a Zouave second lieutenant – Paul-Eugène Milliet – and painted boats on the sea and the village. MacKnight introduced Van Gogh to Eugène Boch, a Belgian painter who sometimes stayed in Fontvieille, and the two exchanged visits in July.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t==== Gauguin's visit (1888) ==== When Gauguin agreed to visit Arles in 1888, Van Gogh hoped for friendship and to realise his idea of an artists' collective. Van Gogh prepared for Gauguin's arrival by painting four versions of Sunflowers in one week. ""In the hope of living in a studio of our own with Gauguin,"" he wrote in a letter to Theo, ""I'd like to do a decoration for the studio. Nothing but large Sunflowers."" When Boch visited again, Van Gogh painted a portrait of him, as well as the study The Poet Against a Starry Sky.In preparation for Gauguin's visit, Van Gogh bought two beds on advice from the station's postal supervisor Joseph Roulin, whose portrait he painted. On 17 September, he spent his first night in the still sparsely furnished Yellow House. When Gauguin consented to work and live in Arles with him, Van Gogh started to work on the Décoration for the Yellow House, probably the most ambitious effort he ever undertook. He completed two chair paintings: Van Gogh's Chair and Gauguin's Chair.After much pleading from Van Gogh, Gauguin arrived in Arles on 23 October and, in November, the two painted together. Gauguin depicted Van Gogh in his The Painter of Sunflowers; Van Gogh painted pictures from memory, following Gauguin's suggestion. Among these ""imaginative"" paintings is Memory of the Garden at Etten. Their first joint outdoor venture was at the Alyscamps, when they produced the pendants Les Alyscamps. The single painting Gauguin completed during his visit was his portrait of Van Gogh."
1,10.0,"When Gauguin consented to work and live in Arles with him, Van Gogh started to work on the Décoration for the Yellow House, probably the most ambitious effort he ever undertook. He completed two chair paintings: Van Gogh's Chair and Gauguin's Chair.After much pleading from Van Gogh, Gauguin arrived in Arles on 23 October and, in November, the two painted together. Gauguin depicted Van Gogh in his The Painter of Sunflowers; Van Gogh painted pictures from memory, following Gauguin's suggestion. Among these ""imaginative"" paintings is Memory of the Garden at Etten. Their first joint outdoor venture was at the Alyscamps, when they produced the pendants Les Alyscamps. The single painting Gauguin completed during his visit was his portrait of Van Gogh.Van Gogh and Gauguin visited Montpellier in December 1888, where they saw works by Courbet and Delacroix in the Musée Fabre. Their relationship began to deteriorate; Van Gogh admired Gauguin and wanted to be treated as his equal, but Gauguin was arrogant and domineering, which frustrated Van Gogh. They often quarrelled; Van Gogh increasingly feared that Gauguin was going to desert him, and the situation, which Van Gogh described as one of ""excessive tension"", rapidly headed towards crisis point.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t==== Hospital in Arles (December 1888) ====The exact sequence that led to the mutilation of Van Gogh's ear is not known. Gauguin said, fifteen years later, that the night followed several instances of physically threatening behaviour. Their relationship was complex and Theo may have owed money to Gauguin, who suspected the brothers were exploiting him financially. It seems likely that Vincent realised that Gauguin was planning to leave. The following days saw heavy rain, leading to the two men being shut in the Yellow House. Gauguin recalled that Van Gogh followed him after he left for a walk and ""rushed towards me, an open razor in his hand""."
2,7.0,"Later he drifted into ill-health and